In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [43]:
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed
from tensorflow.keras.utils import to_categorical
tensorflow.config.run_functions_eagerly(True)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "C:/Users/Hp/Downloads/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.current_dictionary = self.full_dictionary.copy()

        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.tokenizer = Tokenizer(char_level = True)
        self.model = self.cr_model()
    
    def create_masked_data(self, word, num_masks):
        word_len = len(word)
        mask_positions = random.sample(range(word_len), num_masks)
        masked_word = ''.join([word[i] if i not in mask_positions else '_' for i in range(word_len)])
        target_letters = [word[i] for i in mask_positions]
    
        return masked_word, target_letters

    def generate_dataset(self, word_dataset):
        X_data = []
        y_data = []
    
        for word in word_dataset:
            word_len = len(word)
            for length in (1, word_len):
                num_masks = length
                masked_word, target_letters = self.create_masked_data(word, num_masks)
                X_data.append(masked_word)
                y_data.append(target_letters)
        
        self.tokenizer.fit_on_texts(X_data)
        X_sequences = self.tokenizer.texts_to_sequences(X_data)
        y_sequences = self.tokenizer.texts_to_sequences([''.join(y) for y in y_data])
        max_len = max(len(seq) for seq in X_sequences)
        X_padded = pad_sequences(X_sequences, maxlen=max_len, padding='post')
        y_padded = pad_sequences(y_sequences, maxlen=max_len, padding='post')
        
        return X_padded, y_padded

    def cr_model(self):
        embedding_dim = 64
        lstm_units = 128
        X_padded, y_padded = self.generate_dataset(self.current_dictionary)
        max_len = max(len(seq) for seq in X_padded)
        vocab_size = len(self.tokenizer.word_index) + 1
        self.model = Sequential()
        self.model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len, mask_zero=True))
        self.model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
        self.model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        y_categorical = [to_categorical(seq, num_classes=vocab_size) for seq in y_padded]
        X_train = np.array(X_padded)
        y_train = np.array(y_categorical)
        self.model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

        return self.model

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        ns_word = word.replace(" ", "")
        word_sequence = self.tokenizer.texts_to_sequences([ns_word])
        word_padded = pad_sequences(word_sequence, maxlen=self.model.input_shape[1], padding='post')

        predictions = self.model.predict(word_padded)
        predicted_sequence = [np.argmax(pred) for pred in predictions[0]]
        predicted_word = ''.join([self.tokenizer.index_word.get(i, '_') for i in predicted_sequence])
        for letter in predicted_word:
            if letter not in self.guessed_letters:
                self.guessed_letters.append(letter)
                return letter
        for letter in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                self.guessed_letters.append(letter)
                return letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [44]:
api = HangmanAPI(access_token="6fe15a44f329635503c85ea2c532fb", timeout=2000)

C:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 24548s 4s/step - accuracy: 0.2333 - loss: 1.6350 - val_accuracy: 0.1707 - val_loss: 1.6351
Epoch 2/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 11960s 2s/step - accuracy: 0.1657 - loss: 1.5680 - val_accuracy: 0.1724 - val_loss: 1.6261
Epoch 3/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5778s 1s/step - accuracy: 0.1668 - loss: 1.5564 - val_accuracy: 0.1725 - val_loss: 1.6294
Epoch 4/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5714s 1s/step - accuracy: 0.1675 - loss: 1.5493 - val_accuracy: 0.1732 - val_loss: 1.6288
Epoch 5/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5703s 1s/step - accuracy: 0.1681 - loss: 1.5457 - val_accuracy: 0.1731 - val_loss: 1.6287
Epoch 6/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5689s 1s/step - accuracy: 0.1680 - loss: 1.5438 - val_accuracy: 0.1732 - val_loss: 1.6270
Epoch 7/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5691s 1s/step - accuracy: 0.1692 - loss: 1.5343 - val_accuracy: 0.1729 - val_loss: 1.6296
Epoch 8/10
5683/5683 ━━━━━━━━━━━━━━━━━━━━ 5585s 983ms/step - accuracy: 0.1

In [54]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 1168e1d354f4. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
Guessing letter: e
Sever response: {'game_id': '1168e1d354f4', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
Guessing letter: l
Sever response: {'game_id': '1168e1d354f4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ e _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
Guessing letter: a
Sever response: {'game_id': '1168e1d354f4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ a e _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
Guessing letter: s
Sever response: {'game_id': '1168e1d354f4', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a e _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step
Guessing letter: _
Sever response: {'game_id': '1168e1d354f4', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ a e _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
Guessing letter

In [59]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  836  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Playing  837  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
Playing  838  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/ste

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}